In [1]:
import pandas as pd

In [5]:
# Load the actionshistory
data = pd.read_json('/eos/user/l/llayer/AIErrorHandling/actionshistory.json', orient='index')

# Reset index
data_index_reset = data.reset_index()
data_index_reset = data_index_reset.rename(columns={'index': 'task_name'})

In [6]:
data_index_reset.head()

,task_name,errors,parameters
0,/amaltaro_Run2016D-v2-DoubleMuonLowMass-07Aug1...,"{u'good_sites': {}, u'bad_sites': {u'-1': {u'T...","{u'action': u'acdc', u'sites': [u'T1_US_FNAL']..."
1,/amaltaro_Run2016D-v2-DoubleMuonLowMass-07Aug1...,"{u'good_sites': {}, u'bad_sites': {u'-1': {u'T...","{u'action': u'acdc', u'sites': [u'T1_US_FNAL']..."
2,/amaltaro_Run2016D-v2-DoubleMuonLowMass-07Aug1...,"{u'good_sites': {}, u'bad_sites': {u'-1': {u'T...","{u'action': u'acdc', u'sites': [u'T1_US_FNAL']..."
3,/amaltaro_Run2018A-v1-DoubleMuon-17Sep2018_102...,"{u'good_sites': {u'85': {u'T1_UK_RAL': 1}}, u'...","{u'action': u'acdc', u'cores': u'', u'xrootd':..."
4,/amaltaro_Run2018A-v1-DoubleMuon-17Sep2018_102...,"{u'good_sites': {u'-1': {u'T2_DE_RWTH': 1, u'T...","{u'action': u'acdc', u'cores': u'', u'sites': ..."


In [40]:
def loop_over_sites(sites, task_name):
    key = []
    for exit_code, site_dict in zip(sites.keys(), sites.values()):
        if int(exit_code) == -1:
            continue
        for site in site_dict:
            key.append((task_name, exit_code, site))
    return key
        

In [65]:
def map_to_key(row):
    
    task_name = row['task_name']
    errors = row['errors']
    good_sites = errors['good_sites']
    bad_sites = errors['bad_sites']
    
    good_sites_key = loop_over_sites(good_sites, task_name)
    bad_sites_key = loop_over_sites(bad_sites, task_name)
    #return list(set(good_sites_key + bad_sites_key))
    if len(list(set(good_sites_key + bad_sites_key))) < len(good_sites_key + bad_sites_key):
        print
        print good_sites_key
        print bad_sites_key
        print
        
    return list(set(good_sites_key + bad_sites_key))

In [66]:
data_index_reset['non_zero_keys'] = data_index_reset.apply(lambda x: map_to_key(x), axis = 1)

In [67]:
data_index_reset['non_zero_keys'].head(100)

0                                                    []
1                                                    []
2                                                    []
3     [(/amaltaro_Run2018A-v1-DoubleMuon-17Sep2018_1...
4     [(/amaltaro_Run2018A-v1-DoubleMuon-17Sep2018_1...
5     [(/amaltaro_Run2018A-v1-DoubleMuon-17Sep2018_1...
6                                                    []
7                                                    []
8     [(/amaltaro_Run2018A-v1-DoubleMuon-17Sep2018_1...
9     [(/amaltaro_Run2018A-v1-DoubleMuon-17Sep2018_1...
10                                                   []
11    [(/amaltaro_Run2018A-v1-EGamma-17Sep2018_1024p...
12                                                   []
13                                                   []
14                                                   []
15    [(/amaltaro_Run2018A-v1-EGamma-17Sep2018_1024p...
16                                                   []
17                                              

In [68]:
keys = data_index_reset['non_zero_keys'].apply(pd.Series).unstack().reset_index(drop=True).dropna()

In [69]:
print len(keys)

114134
